*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
df1 = pd.read_csv('/content/train-data.tsv', sep='\t', names=["label", "message"])

df2 = pd.read_csv('/content/valid-data.tsv', sep='\t', names=["label", "message"])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
ham_msg = df1[df1.label == "ham"]
spam_msg = df1[df1.label == "spam"]

ham_msg_txt = "".join(ham_msg.message.to_numpy().tolist())
spam_msg_txt = "".join(spam_msg.message.to_numpy().tolist())

In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
%matplotlib inline
ham_msg_cloud = WordCloud(width =520, height =260, stopwords=STOPWORDS, max_font_size=50, background_color="black", colormap="Blues").generate(ham_msg_txt)
plt.figure(figsize=(16,10))
plt.imshow(ham_msg_cloud, interpolation='bilinear')
plt.axis('off') # turn off axis
plt.show()

In [ ]:
spam_msg_cloud = WordCloud(width =520, height =260, stopwords=STOPWORDS,max_font_size=50, background_color ="black", colormap='Blues').generate(spam_msg_txt)
plt.figure(figsize=(16,10))
plt.imshow(spam_msg_cloud, interpolation='bilinear')
plt.axis('off') # turn off axis
plt.show()

In [ ]:
ham_msg

In [ ]:

ham_msg_df = ham_msg.sample(n = len(spam_msg), random_state = 44)
spam_msg_df = spam_msg
print(ham_msg_df.shape, spam_msg_df.shape)

In [ ]:

msg_df = ham_msg_df.append(spam_msg_df).reset_index(drop=True)

In [ ]:
msg_df['label']= msg_df['label'].map({'ham': 0, 'spam': 1})

In [ ]:
msg_df


In [ ]:
df2['label']= df2['label'].map({'ham': 0, 'spam': 1})

In [ ]:
msg1=msg_df['message'].values
msg2=df2['message'].values

In [ ]:
msg1

In [ ]:
max_len = 70
trunc_type = "post" 
padding_type = "post" 
oov_tok = "<OOV>" 
vocab_size = 500

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(num_words = vocab_size, char_level=False, oov_token = oov_tok)
tokenizer.fit_on_texts(msg_df['message'])

In [ ]:
word_index = tokenizer.word_index
word_index

In [ ]:
tot_words = len(word_index)
print('There are %s unique tokens in training data. ' % tot_words)

In [ ]:
training_sequences = tokenizer.texts_to_sequences(msg_df['message'])
training_padded = pad_sequences (training_sequences, maxlen = max_len, padding = padding_type, truncating = trunc_type )
testing_sequences = tokenizer.texts_to_sequences(df2['message'])
testing_padded = pad_sequences(testing_sequences, maxlen = max_len,padding = padding_type, truncating = trunc_type)

In [ ]:

print(training_padded[1])

In [ ]:
print('Shape of training tensor: ', training_padded.shape)
print('Shape of testing tensor: ', testing_padded.shape)

In [ ]:
vocab_size = len(tokenizer.word_index) + 1 # As defined earlier
embeding_dim = 24
drop_value = 0.2 # dropout
n_dense = 24

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense, Dropout, LSTM, Bidirectional
model = Sequential()
model.add(Embedding(vocab_size, embeding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(24, activation='relu'))
model.add(Dropout(drop_value))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam' ,metrics=['accuracy'])

In [ ]:

model.summary()

In [ ]:

num_epochs = 30
early_stop = EarlyStopping(monitor='val_loss', patience=3)
history = model.fit(training_padded, msg_df['label'], epochs=num_epochs, validation_data=(testing_padded, df2['label']),callbacks =[early_stop], verbose=2)

In [ ]:
model.evaluate(testing_padded, df2['label'])

In [ ]:
n_lstm = 20
drop_lstm =0.2
model1 = Sequential()
model1.add(Embedding(vocab_size, embeding_dim, input_length=max_len))
model1.add(LSTM(n_lstm, dropout=drop_lstm, return_sequences=True))
model1.add(LSTM(n_lstm, dropout=drop_lstm, return_sequences=True))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])

In [ ]:
num_epochs = 30
early_stop = EarlyStopping(monitor='val_loss', patience=2)
history = model1.fit(training_padded, msg_df['label'], epochs=num_epochs, validation_data=(testing_padded, df2['label']),callbacks =[early_stop], verbose=2)

In [ ]:
model1.evaluate(testing_padded, df2['label'])

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    
    
    new_seq = tokenizer.texts_to_sequences([pred_text])
    pad = pad_sequences (new_seq, maxlen = max_len, padding = padding_type, truncating = trunc_type )
  

   
    answer=model.predict(pad)

    #prediction=[]
    result=""
    if answer[0]*100<5:
        result="ham"
    else:
        result="spam"

    #prediction.append(result)
   



    return (answer,result)

pred_text = "ball is not good"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:


# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    
    
    new_seq = tokenizer.texts_to_sequences([pred_text])
    pad = pad_sequences (new_seq, maxlen = max_len, padding = padding_type, truncating = trunc_type )
  

   
    answer=model.predict(pad)

    #prediction=[]
    result=""
    if answer[0]*100<5:
        result="ham"
    else:
        result="spam"

    #prediction.append(result)
   



    return (answer,result)

pred_text = "ball is not good"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
